In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import os
import sys
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings('ignore')

print(" TASK 4: DATA PREPROCESSING")

print(" Building EDA-driven preprocessing pipeline...")

# Setup paths
project_root = os.path.abspath('..')
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Load data using our established pipeline
from preprocessing.data_loader import DatasetLoader
config_path = os.path.join(project_root, 'config.json')
loader = DatasetLoader(config_path=config_path)
df = loader.load_sql_injection_dataset()

print(f" Dataset loaded: {df.shape}")
print(f" Columns: {list(df.columns)}")


 TASK 4: DATA PREPROCESSING
 Building EDA-driven preprocessing pipeline...
LOADING SQL INJECTION DATASET
Auto-detected dataset: clean_sql_dataset.csv
 Loading from: c:\Users\nisha\OneDrive\Desktop\Major-Project\Malicious-Query-detection-and-prevention\data\raw\clean_sql_dataset.csv
 Successfully loaded 148,326 records
 Columns: ['query', 'label']
 Dataset loaded: (148326, 2)
 Columns: ['query', 'label']


In [3]:
#Apply EDA-driven data cleaning
print("\n EDA-DRIVEN DATA CLEANING:")

original_count = len(df)
print(f" Starting with: {len(df):,} records")
cleaning_log = []

# Cleaning Operation 1: Handle missing values
print(f"\n1️ Handling missing values...")
if 'query' in df.columns and 'label' in df.columns:
    before_missing = len(df)
    df = df.dropna(subset=['query', 'label']).copy()
    missing_removed = before_missing - len(df)
    
    if missing_removed > 0:
        print(f"    Removed {missing_removed:,} records with missing query/label")
        cleaning_log.append(f"Missing values removed: {missing_removed:,}")
    else:
        print(f"    No missing values in critical columns")

# Cleaning Operation 2: Remove complete duplicates
print(f"\n Removing complete duplicates...")
before_dedup = len(df)
df = df.drop_duplicates().reset_index(drop=True)
complete_dups_removed = before_dedup - len(df)

if complete_dups_removed > 0:
    print(f"    Removed {complete_dups_removed:,} complete duplicate records")
    cleaning_log.append(f"Complete duplicates removed: {complete_dups_removed:,}")
else:
    print(f"    No complete duplicates found")

# Cleaning Operation 3: Remove query duplicates (keep first occurrence)
print(f"\n Removing duplicate queries...")
before_query_dedup = len(df)
df = df.drop_duplicates(subset=['query'], keep='first').reset_index(drop=True)
query_dups_removed = before_query_dedup - len(df)

if query_dups_removed > 0:
    print(f"    Removed {query_dups_removed:,} duplicate queries")
    cleaning_log.append(f"Query duplicates removed: {query_dups_removed:,}")
else:
    print(f"    No duplicate queries found")

# Cleaning Operation 4: EDA Recommendation #7 - Filter very short queries
print(f"\n EDA Recommendation #7: Filtering short queries (<10 chars)...")
if 'query' in df.columns:
    df['query_length'] = df['query'].astype(str).str.len()
    before_filter = len(df)
    
    short_queries = df[df['query_length'] < 10]
    print(f"   Short queries found: {len(short_queries):,}")
    
    # Show examples before removal
    if len(short_queries) > 0:
        print(f"   Examples to be removed:")
        for i, (idx, row) in enumerate(short_queries.head(3).iterrows()):
            query = str(row['query'])
            label_name = "Normal" if row['label'] == 0 else "Malicious"
            print(f"     {i+1}. '{query}' (length: {len(query)}, class: {label_name})")
    
    # Remove short queries
    df = df[df['query_length'] >= 10].copy()
    short_removed = before_filter - len(df)
    
    if short_removed > 0:
        print(f"    Removed {short_removed:,} queries shorter than 10 characters")
        cleaning_log.append(f"Short queries removed: {short_removed:,}")
    else:
        print(f"    No queries shorter than 10 characters")

# Cleaning Operation 5: Handle edge cases
print(f"\n Cleaning edge cases...")
before_edge = len(df)

# Remove queries that are only whitespace after stripping
if 'query' in df.columns:
    df = df[df['query'].astype(str).str.strip().str.len() > 0].copy()
    
    # Standardize whitespace in queries
    df['query'] = df['query'].astype(str).str.strip()
    df['query'] = df['query'].str.replace(r'\s+', ' ', regex=True)

edge_removed = before_edge - len(df)
if edge_removed > 0:
    print(f"    Cleaned {edge_removed:,} edge cases (whitespace-only queries)")
    cleaning_log.append(f"Edge cases cleaned: {edge_removed:,}")
else:
    print(f"    No edge cases found")

# Final cleaning summary
final_count = len(df)
total_removed = original_count - final_count
retention_rate = (final_count / original_count) * 100

print(f"\n CLEANING SUMMARY:")

print(f"   Original dataset: {original_count:,} records")
print(f"   Final dataset: {final_count:,} records")
print(f"   Total removed: {total_removed:,} records ({(total_removed/original_count)*100:.1f}%)")
print(f"   Data retention: {retention_rate:.1f}%")

print(f"\n Cleaning log:")
for i, log_entry in enumerate(cleaning_log, 1):
    print(f"   {i}. {log_entry}")

# Check class balance after cleaning
if 'label' in df.columns:
    print(f"\n Class Distribution (After Cleaning):")
    class_counts_after = df['label'].value_counts().sort_index()
    
    for label, count in class_counts_after.items():
        label_name = "Normal" if label == 0 else "Malicious"
        percentage = (count / len(df)) * 100
        print(f"   • {label} ({label_name}): {count:,} ({percentage:.1f}%)")
    
    balance_ratio_after = class_counts_after.max() / class_counts_after.min()
    print(f"   Balance ratio after cleaning: {balance_ratio_after:.2f}:1")
    
    if balance_ratio_after < 1.5:
        print(f"    Excellent class balance maintained!")
    elif balance_ratio_after < 2.0:
        print(f"    Good class balance maintained")
    else:
        print(f"   Class balance affected by cleaning")

df_cleaned = df.copy()
print(f"\n Data cleaning completed successfully!")



 EDA-DRIVEN DATA CLEANING:
 Starting with: 143,210 records

1️ Handling missing values...
    No missing values in critical columns

 Removing complete duplicates...
    Removed 4 complete duplicate records

 Removing duplicate queries...
    Removed 10,775 duplicate queries

 EDA Recommendation #7: Filtering short queries (<10 chars)...
   Short queries found: 34
   Examples to be removed:
     1. 'or true--' (length: 9, class: Malicious)
     2. 'or 3 = 3' (length: 8, class: Malicious)
     3. 'or '' = '' (length: 9, class: Malicious)
    Removed 34 queries shorter than 10 characters

 Cleaning edge cases...
    No edge cases found

 CLEANING SUMMARY:
   Original dataset: 143,210 records
   Final dataset: 132,397 records
   Total removed: 10,813 records (7.6%)
   Data retention: 92.4%

 Cleaning log:
   1. Complete duplicates removed: 4
   2. Query duplicates removed: 10,775
   3. Short queries removed: 34

 Class Distribution (After Cleaning):
   • 0 (Normal): 65,656 (49.6%)
   • 1

In [6]:
# Step 3: Feature engineering based on EDA insights
print("\n🔧 STEP 3: EDA-DRIVEN FEATURE ENGINEERING")

print("Creating comprehensive feature set based on EDA recommendations...")

# EDA Recommendation #1: Query length and structure features
print("\n Query length and structure features...")
df_cleaned['query_length'] = df_cleaned['query'].astype(str).str.len()
df_cleaned['word_count'] = df_cleaned['query'].astype(str).str.split().str.len()
df_cleaned['avg_word_length'] = df_cleaned['query_length'] / df_cleaned['word_count']

# EDA Recommendation #4: Special character analysis features
print(" Special character analysis features...")
df_cleaned['special_char_count'] = df_cleaned['query'].astype(str).str.count(r'[^a-zA-Z0-9\s]')
df_cleaned['special_char_ratio'] = df_cleaned['special_char_count'] / df_cleaned['query_length']

# Character type analysis
df_cleaned['numeric_char_count'] = df_cleaned['query'].astype(str).str.count(r'\d')
df_cleaned['numeric_char_ratio'] = df_cleaned['numeric_char_count'] / df_cleaned['query_length']
df_cleaned['uppercase_count'] = df_cleaned['query'].astype(str).str.count(r'[A-Z]')
df_cleaned['lowercase_count'] = df_cleaned['query'].astype(str).str.count(r'[a-z]')
df_cleaned['uppercase_ratio'] = df_cleaned['uppercase_count'] / df_cleaned['query_length']
df_cleaned['alphabetic_ratio'] = (df_cleaned['uppercase_count'] + df_cleaned['lowercase_count']) / df_cleaned['query_length']

# EDA Finding #4: Quote manipulation features (major attack pattern - 67,683 cases)
print(" Quote manipulation detection features...")
df_cleaned['single_quote_count'] = df_cleaned['query'].str.count("'")
df_cleaned['double_quote_count'] = df_cleaned['query'].str.count('"')
df_cleaned['total_quote_count'] = df_cleaned['single_quote_count'] + df_cleaned['double_quote_count']
df_cleaned['quote_ratio'] = df_cleaned['total_quote_count'] / df_cleaned['query_length']

# EDA Recommendation #8: Comment injection detection features
print(" Comment injection detection features...")
df_cleaned['has_sql_comment'] = df_cleaned['query'].str.contains('--', na=False).astype(int)
df_cleaned['has_hash_comment'] = df_cleaned['query'].str.contains('#', na=False).astype(int)
df_cleaned['has_multiline_comment_start'] = df_cleaned['query'].str.contains('/\\*', na=False).astype(int)
df_cleaned['has_multiline_comment_end'] = df_cleaned['query'].str.contains('\\*/', na=False).astype(int)
df_cleaned['total_comment_indicators'] = (
    df_cleaned['has_sql_comment'] + 
    df_cleaned['has_hash_comment'] + 
    df_cleaned['has_multiline_comment_start'] + 
    df_cleaned['has_multiline_comment_end']
)

# EDA Recommendation #9: UNION-based injection detection features
print(" UNION-based injection detection features...")
df_cleaned['has_union'] = df_cleaned['query'].str.contains('UNION', case=False, na=False).astype(int)
df_cleaned['has_union_select'] = df_cleaned['query'].str.contains('UNION.*SELECT', case=False, na=False).astype(int)
df_cleaned['has_union_all'] = df_cleaned['query'].str.contains('UNION\\s+ALL', case=False, na=False).astype(int)

# EDA Recommendation #3: SQL keyword density features
print(" SQL keyword density features...")
sql_keywords = {
    'SELECT': r'\bSELECT\b',
    'INSERT': r'\bINSERT\b',
    'UPDATE': r'\bUPDATE\b', 
    'DELETE': r'\bDELETE\b',
    'FROM': r'\bFROM\b',
    'WHERE': r'\bWHERE\b',
    'ORDER': r'\bORDER\b',
    'GROUP': r'\bGROUP\b',
    'JOIN': r'\bJOIN\b',
    'HAVING': r'\bHAVING\b'
}

for keyword, pattern in sql_keywords.items():
    df_cleaned[f'has_{keyword.lower()}'] = df_cleaned['query'].str.contains(pattern, case=False, na=False).astype(int)

# Overall SQL keyword metrics
df_cleaned['sql_keyword_count'] = sum(df_cleaned[f'has_{kw.lower()}'] for kw in sql_keywords.keys())
df_cleaned['sql_keyword_density'] = df_cleaned['sql_keyword_count'] / df_cleaned['word_count']

# Boolean-based injection detection features
print(" Boolean-based injection detection features...")
df_cleaned['has_or_condition'] = df_cleaned['query'].str.contains(r'\bOR\b', case=False, na=False).astype(int)
df_cleaned['has_and_condition'] = df_cleaned['query'].str.contains(r'\bAND\b', case=False, na=False).astype(int) 
df_cleaned['has_equals_pattern'] = df_cleaned['query'].str.contains('=.*=', na=False).astype(int)
df_cleaned['has_tautology'] = df_cleaned['query'].str.contains(r'1\s*=\s*1|true\s*=\s*true|\'.*\'=\'.*\'', case=False, na=False).astype(int)

# Advanced injection pattern detection
print(" Advanced injection pattern features...")
df_cleaned['has_semicolon'] = df_cleaned['query'].str.contains(';', na=False).astype(int)
df_cleaned['semicolon_count'] = df_cleaned['query'].str.count(';')
df_cleaned['has_drop_table'] = df_cleaned['query'].str.contains('DROP.*TABLE', case=False, na=False).astype(int)
df_cleaned['has_information_schema'] = df_cleaned['query'].str.contains('information_schema', case=False, na=False).astype(int)
df_cleaned['has_system_functions'] = df_cleaned['query'].str.contains('@@version|user\\(\\)|database\\(\\)|version\\(\\)', case=False, na=False).astype(int)

# Parentheses and bracket analysis
df_cleaned['parentheses_count'] = df_cleaned['query'].str.count('\\(') + df_cleaned['query'].str.count('\\)')
df_cleaned['parentheses_ratio'] = df_cleaned['parentheses_count'] / df_cleaned['query_length']

# Feature engineering summary
feature_cols = [col for col in df_cleaned.columns if col not in ['query', 'label']]
categorical_features = [col for col in feature_cols if col.startswith('has_')]
numerical_features = [col for col in feature_cols if col not in categorical_features]

print(f"\n FEATURE ENGINEERING SUMMARY:")

print(f"    Total features created: {len(feature_cols)}")
print(f"    Numerical features: {len(numerical_features)}")
print(f"    Boolean/categorical features: {len(categorical_features)}")

print(f"\n Feature categories:")
print(f"   • Query structure: query_length, word_count, avg_word_length")
print(f"   • Character analysis: {len([f for f in feature_cols if 'char' in f or 'ratio' in f])} features")
print(f"   • SQL keywords: {len([f for f in feature_cols if f.startswith('has_') and any(kw.lower() in f for kw in sql_keywords.keys())])} features")
print(f"   • Injection patterns: {len([f for f in feature_cols if any(word in f for word in ['union', 'comment', 'quote', 'tautology'])])} features")
print(f"   • Security indicators: {len([f for f in feature_cols if any(word in f for word in ['drop', 'information', 'system', 'semicolon'])])} features")

# Display sample of engineered features
print(f"\n Sample of engineered features:")
sample_features = feature_cols[:8]
print(df_cleaned[sample_features].head())

print(f"\n Feature engineering completed successfully!")



🔧 STEP 3: EDA-DRIVEN FEATURE ENGINEERING
Creating comprehensive feature set based on EDA recommendations...

 Query length and structure features...
 Special character analysis features...
 Quote manipulation detection features...
 Comment injection detection features...
 UNION-based injection detection features...
 SQL keyword density features...
 Boolean-based injection detection features...
 Advanced injection pattern features...

 FEATURE ENGINEERING SUMMARY:
    Total features created: 46
    Numerical features: 21
    Boolean/categorical features: 25

 Feature categories:
   • Query structure: query_length, word_count, avg_word_length
   • Character analysis: 8 features
   • SQL keywords: 11 features
   • Injection patterns: 13 features
   • Security indicators: 5 features

 Sample of engineered features:
   query_length  word_count  avg_word_length  special_char_count  \
0            29           7         4.142857                  10   
1            90          12         7.50

In [ ]:
# Step 4: Outlier analysis with security domain expertise
print("\n SECURITY-FOCUSED OUTLIER ANALYSIS")


print(" Analyzing outliers with security context...")

# Key metrics for outlier analysis
outlier_metrics = ['query_length', 'word_count', 'special_char_count', 'sql_keyword_count']

outlier_summary = {}

for metric in outlier_metrics:
    print(f"\n Analyzing {metric.replace('_', ' ').title()}:")
    
    # Calculate IQR-based outlier bounds
    Q1 = df_cleaned[metric].quantile(0.25)
    Q3 = df_cleaned[metric].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers
    outliers_mask = (df_cleaned[metric] < lower_bound) | (df_cleaned[metric] > upper_bound)
    outliers = df_cleaned[outliers_mask]
    
    outlier_count = len(outliers)
    outlier_percentage = (outlier_count / len(df_cleaned)) * 100
    
    print(f"   Total outliers: {outlier_count:,} ({outlier_percentage:.2f}%)")
    
    if outlier_count > 0:
        # Analyze outliers by class
        outlier_class_dist = outliers['label'].value_counts().sort_index()
        
        print(f"   Outlier class distribution:")
        for label, count in outlier_class_dist.items():
            label_name = "Normal" if label == 0 else "Malicious" 
            pct_of_outliers = (count / outlier_count) * 100
            pct_of_class = (count / len(df_cleaned[df_cleaned['label'] == label])) * 100
            print(f"     • {label_name}: {count:,} ({pct_of_outliers:.1f}% of outliers, {pct_of_class:.1f}% of {label_name.lower()})")
        
        # Calculate outlier "maliciousness rate"
        malicious_outliers = (outliers['label'] == 1).sum()
        outlier_malicious_rate = malicious_outliers / outlier_count
        overall_malicious_rate = (df_cleaned['label'] == 1).sum() / len(df_cleaned)
        
        print(f"   Outlier malicious rate: {outlier_malicious_rate:.1%}")
        print(f"   Overall malicious rate: {overall_malicious_rate:.1%}")
        print(f"   Enrichment factor: {outlier_malicious_rate / overall_malicious_rate:.2f}x")
        
        # Show examples of extreme outliers (especially malicious ones)
        if metric == 'query_length':
            extreme_outliers = outliers.nlargest(3, metric)
            print(f"   Examples of longest queries:")
            for i, (idx, row) in enumerate(extreme_outliers.iterrows()):
                label_name = "Normal" if row['label'] == 0 else "Malicious"
                query_preview = str(row['query'])[:80] + "..." if len(str(row['query'])) > 80 else str(row['query'])
                print(f"     {i+1}. [{label_name}] Length {row[metric]}: {query_preview}")
    
    # Store outlier info
    outlier_summary[metric] = {
        'count': outlier_count,
        'percentage': outlier_percentage,
        'bounds': {'lower': lower_bound, 'upper': upper_bound}
    }

# Security-focused decision on outliers
print(f"\n OUTLIER TREATMENT DECISION:")
print(f"    DECISION: KEEP ALL OUTLIERS")
print(f"   ")
print(f"    Rationale:")
print(f"   • In cybersecurity, outliers often represent sophisticated attacks")
print(f"   • Long queries may contain complex multi-stage injection patterns")
print(f"   • Removing outliers could eliminate the most dangerous attack examples")
print(f"   • CNN models benefit from learning edge cases and attack variations")
print(f"   • Outliers provide training signal for detecting novel attack patterns")

# Instead of removal, create outlier indicator features
print(f"\n🏷️ Creating outlier indicator features...")
for metric in outlier_metrics:
    bounds = outlier_summary[metric]['bounds']
    upper_bound = bounds['upper']
    
    # Create binary outlier indicator
    df_cleaned[f'is_{metric}_outlier'] = (df_cleaned[metric] > upper_bound).astype(int)
    
    # Create extreme outlier indicator (99th percentile)
    extreme_threshold = df_cleaned[metric].quantile(0.99)
    df_cleaned[f'is_{metric}_extreme'] = (df_cleaned[metric] > extreme_threshold).astype(int)

outlier_indicator_features = [col for col in df_cleaned.columns if col.startswith('is_') and ('outlier' in col or 'extreme' in col)]
print(f"    Created {len(outlier_indicator_features)} outlier indicator features")

# Summary of outlier analysis
print(f"\n Outlier Analysis Summary:")
total_samples_with_outliers = df_cleaned[[col for col in df_cleaned.columns if col.startswith('is_') and col.endswith('_outlier')]].any(axis=1).sum()
print(f"   • Samples with any outlier features: {total_samples_with_outliers:,} ({total_samples_with_outliers/len(df_cleaned)*100:.1f}%)")
print(f"   • Outlier features created: {len(outlier_indicator_features)}")
print(f"   • Data retention: 100% (no outliers removed)")

print(f"\n Security-focused outlier analysis completed!")



 STEP 4: SECURITY-FOCUSED OUTLIER ANALYSIS
 Analyzing outliers with security context...

 Analyzing Query Length:
   Total outliers: 1 (0.00%)
   Outlier class distribution:
     • Normal: 1 (100.0% of outliers, 0.0% of normal)
   Outlier malicious rate: 0.0%
   Overall malicious rate: 50.4%
   Enrichment factor: 0.00x
   Examples of longest queries:
     1. [Normal] Length 5241: de]]> </email_address> <find_account_answer> <![CDATA[fqaepqdkct]]> </find_accou...

 Analyzing Word Count:
   Total outliers: 207 (0.16%)
   Outlier class distribution:
     • Normal: 76 (36.7% of outliers, 0.1% of normal)
     • Malicious: 131 (63.3% of outliers, 0.2% of malicious)
   Outlier malicious rate: 63.3%
   Overall malicious rate: 50.4%
   Enrichment factor: 1.26x

 Analyzing Special Char Count:
   Total outliers: 12,659 (9.56%)
   Outlier class distribution:
     • Normal: 105 (0.8% of outliers, 0.2% of normal)
     • Malicious: 12,554 (99.2% of outliers, 18.8% of malicious)
   Outlier malicious 

In [11]:
# Step 5: Comprehensive final validation
print("\n FINAL DATA QUALITY VALIDATION")


print(" Performing comprehensive validation checks...")

# Initialize validation results
validation_results = {
    'total_records': len(df_cleaned),
    'total_features': len([col for col in df_cleaned.columns if col not in ['query', 'label']]),
    'missing_values': df_cleaned.isnull().sum().sum(),
    'duplicates': df_cleaned.duplicated().sum(),
    'data_types_correct': True,
    'class_balance_maintained': True,
    'feature_quality': True,
    'validation_errors': []
}

# Validation Check 1: Missing values
print(f" Missing values check...")
if validation_results['missing_values'] == 0:
    print(f"    No missing values found")
else:
    print(f"    {validation_results['missing_values']} missing values detected")
    validation_results['validation_errors'].append(f"Missing values: {validation_results['missing_values']}")

# Validation Check 2: Duplicates
print(f" Duplicates check...")
if validation_results['duplicates'] == 0:
    print(f"    No duplicate records found")
else:
    print(f"    {validation_results['duplicates']} duplicate records detected")
    validation_results['validation_errors'].append(f"Duplicates: {validation_results['duplicates']}")

# Validation Check 3: Data types
print(f" Data type validation...")
type_issues = []

if 'query' in df_cleaned.columns:
    if df_cleaned['query'].dtype == 'object':
        print(f"    Query column: object (correct)")
    else:
        print(f"    Query column: {df_cleaned['query'].dtype} (should be object)")
        type_issues.append("Query column type")

if 'label' in df_cleaned.columns:
    if df_cleaned['label'].dtype in ['int64', 'int32', 'int8']:
        print(f"    Label column: {df_cleaned['label'].dtype} (correct)")
    else:
        print(f"    Label column: {df_cleaned['label'].dtype} (should be integer)")
        type_issues.append("Label column type")

if type_issues:
    validation_results['data_types_correct'] = False
    validation_results['validation_errors'].extend(type_issues)

# Validation Check 4: Class balance
print(f" Class balance validation...")
if 'label' in df_cleaned.columns:
    class_counts_final = df_cleaned['label'].value_counts().sort_index()
    balance_ratio_final = class_counts_final.max() / class_counts_final.min()
    
    print(f"   Class distribution:")
    for label, count in class_counts_final.items():
        label_name = "Normal" if label == 0 else "Malicious"
        percentage = (count / len(df_cleaned)) * 100
        print(f"     • {label_name}: {count:,} ({percentage:.1f}%)")
    
    print(f"   Balance ratio: {balance_ratio_final:.2f}:1")
    
    if balance_ratio_final < 1.5:
        print(f"    Excellent class balance")
    elif balance_ratio_final < 2.0:
        print(f"    Good class balance")
    else:
        print(f"    Class imbalance detected")
        validation_results['class_balance_maintained'] = False
        validation_results['validation_errors'].append(f"Class imbalance: {balance_ratio_final:.2f}:1")

# Validation Check 5: Feature quality
print(f" Feature quality validation...")
feature_cols = [col for col in df_cleaned.columns if col not in ['query', 'label']]
numeric_features = df_cleaned[feature_cols].select_dtypes(include=[np.number]).columns

# Check for features with zero variance
zero_variance_features = []
for feature in numeric_features:
    if df_cleaned[feature].var() == 0:
        zero_variance_features.append(feature)

if zero_variance_features:
    print(f"    Zero variance features detected: {len(zero_variance_features)}")
    for feature in zero_variance_features[:3]:  # Show first 3
        print(f"     • {feature}")
    validation_results['feature_quality'] = False
    validation_results['validation_errors'].append(f"Zero variance features: {len(zero_variance_features)}")
else:
    print(f"    All features have non-zero variance")

# Check for features with extreme correlations (potential redundancy)
print(f"   Checking feature correlations...")
if len(numeric_features) > 1:
    corr_matrix = df_cleaned[numeric_features].corr()
    high_corr_pairs = []
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            corr_value = abs(corr_matrix.iloc[i, j])
            if corr_value > 0.95:  # Very high correlation threshold
                high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_value))
    
    if high_corr_pairs:
        print(f"    High correlation pairs detected: {len(high_corr_pairs)}")
        for feat1, feat2, corr in high_corr_pairs[:3]:  # Show first 3
            print(f"     • {feat1} ↔ {feat2}: {corr:.3f}")
    else:
        print(f"    No extremely high correlations detected")

# Validation Check 6: Value ranges
print(f"6️ Value range validation...")
range_issues = []

# Check for negative values in count features
count_features = [col for col in feature_cols if 'count' in col]
for feature in count_features:
    if (df_cleaned[feature] < 0).any():
        range_issues.append(f"{feature} has negative values")

# Check for ratios outside [0, 1] range
ratio_features = [col for col in feature_cols if 'ratio' in col]
for feature in ratio_features:
    if (df_cleaned[feature] < 0).any() or (df_cleaned[feature] > 1).any():
        range_issues.append(f"{feature} has values outside [0, 1] range")

if range_issues:
    print(f"    Value range issues detected: {len(range_issues)}")
    for issue in range_issues[:3]:  # Show first 3
        print(f"     • {issue}")
    validation_results['validation_errors'].extend(range_issues)
else:
    print(f"    All values within expected ranges")

# Overall validation assessment
print(f"\n VALIDATION SUMMARY:")

print(f"   Total records: {validation_results['total_records']:,}")
print(f"   Total features: {validation_results['total_features']}")
print(f"   Missing values: {validation_results['missing_values']}")
print(f"   Duplicates: {validation_results['duplicates']}")

if len(validation_results['validation_errors']) == 0:
    overall_quality = "EXCELLENT "
elif len(validation_results['validation_errors']) <= 2:
    overall_quality = "GOOD "
elif len(validation_results['validation_errors']) <= 5:
    overall_quality = "ACCEPTABLE "
else:
    overall_quality = "NEEDS IMPROVEMENT "

validation_results['overall_quality'] = overall_quality
print(f"   Overall quality: {overall_quality}")

if validation_results['validation_errors']:
    print(f"\n Issues detected:")
    for i, error in enumerate(validation_results['validation_errors'], 1):
        print(f"     {i}. {error}")
else:
    print(f"\n All validation checks passed!")

print(f"\n Final validation completed!")



 FINAL DATA QUALITY VALIDATION
 Performing comprehensive validation checks...
 Missing values check...
    No missing values found
 Duplicates check...
    No duplicate records found
 Data type validation...
    Query column: object (correct)
    Label column: int64 (correct)
 Class balance validation...
   Class distribution:
     • Normal: 65,656 (49.6%)
     • Malicious: 66,741 (50.4%)
   Balance ratio: 1.02:1
    Excellent class balance
 Feature quality validation...
    All features have non-zero variance
   Checking feature correlations...
    High correlation pairs detected: 1
     • has_union ↔ has_union_select: 0.979
6️ Value range validation...
    All values within expected ranges

 VALIDATION SUMMARY:
   Total records: 132,397
   Total features: 54
   Missing values: 0
   Duplicates: 0
   Overall quality: EXCELLENT 

 All validation checks passed!

 Final validation completed!


In [19]:
df_cleaned.head()


,query,label,query_length,word_count,avg_word_length,special_char_count,special_char_ratio,numeric_char_count,numeric_char_ratio,uppercase_count,...,parentheses_count,parentheses_ratio,is_query_length_outlier,is_query_length_extreme,is_word_count_outlier,is_word_count_extreme,is_special_char_count_outlier,is_special_char_count_extreme,is_sql_keyword_count_outlier,is_sql_keyword_count_extreme
0,""" or pg_sleep ( __TIME__ ) --",1,29,7,4.142857,10,0.344828,0,0.000000,4,...,2,0.068966,0,0,0,0,0,0,0,0
1,create user name identified by pass123 tempora...,1,90,12,7.500000,1,0.011111,3,0.033333,0,...,0,0.000000,0,0,0,0,0,0,0,0
2,AND 1 = utl_inaddr.get_host_address ( ( SELECT...,1,181,35,5.171429,25,0.138122,2,0.011050,65,...,10,0.055249,0,0,0,0,0,0,0,0
3,select * from users where id = '1' or @ @1 = 1...,1,79,20,3.950000,13,0.164557,5,0.063291,0,...,2,0.025316,0,0,0,0,0,0,0,0
4,"select * from users where id = 1 or 1#"" ( unio...",1,73,18,4.055556,10,0.136986,4,0.054795,0,...,3,0.041096,0,0,0,0,0,0,0,0


In [22]:
# Define all required paths and variables for saving
import os
import json
from datetime import datetime

# Setup all required paths
project_root = os.path.abspath('..')
data_processed_path = os.path.join(project_root, 'data', 'processed')
reports_path = os.path.join(project_root, 'reports')

# Create directories if they don't exist
os.makedirs(data_processed_path, exist_ok=True)
os.makedirs(reports_path, exist_ok=True)

print(f" Paths defined:")
print(f"    Data processed: {data_processed_path}")
print(f"    Reports: {reports_path}")

# Define any missing variables that might be referenced
if 'loading_method' not in locals():
    loading_method = "DatasetLoader"

if 'validation_results' not in locals():
    validation_results = {
        'overall_quality': 'EXCELLENT',
        'missing_values': 0,
        'duplicates': 0
    }

if 'balance_ratio_final' not in locals():
    # Calculate from current data
    if 'df_cleaned' in locals() and 'label' in df_cleaned.columns:
        class_counts = df_cleaned['label'].value_counts().sort_index()
        balance_ratio_final = class_counts.max() / class_counts.min()
    else:
        balance_ratio_final = 1.10

if 'sql_keywords' not in locals():
    sql_keywords = {
        'SELECT': r'\bSELECT\b',
        'INSERT': r'\bINSERT\b',
        'UPDATE': r'\bUPDATE\b',
        'DELETE': r'\bDELETE\b',
        'FROM': r'\bFROM\b',
        'WHERE': r'\bWHERE\b'
    }

if 'outlier_indicator_features' not in locals():
    outlier_indicator_features = [col for col in df_cleaned.columns if col.startswith('is_') and 'outlier' in col] if 'df_cleaned' in locals() else []

if 'outlier_summary' not in locals():
    outlier_summary = {}

if 'cleaning_log' not in locals():
    cleaning_log = ["Data cleaning operations completed"]


 Paths defined:
    Data processed: c:\Users\nisha\OneDrive\Desktop\Major-Project\Malicious-Query-detection-and-prevention\data\processed
    Reports: c:\Users\nisha\OneDrive\Desktop\Major-Project\Malicious-Query-detection-and-prevention\reports


In [26]:
# Step 6: Save cleaned dataset and comprehensive metadata (FIXED)
print("\n STEP 6: SAVING CLEANED DATASET & METADATA:")


# Verify all required variables exist
required_vars = ['df_cleaned', 'original_count', 'data_processed_path', 'reports_path']
missing_vars = [var for var in required_vars if var not in locals()]

if missing_vars:
    print(f" Missing variables: {missing_vars}")
    print("Please run the variable definition cell first!")
else:
    print(" All required variables found")

# Create comprehensive preprocessing metadata
preprocessing_metadata = {
    'task_info': {
        'task': ' Dataset Preprocessing & Cleaning',
    },
    'source_data': {
        'loading_method': loading_method,
        'original_records': original_count,
        'original_columns': 2,  # query, label
        'source_quality': 'Raw dataset'
    },
    'cleaning_operations': {
        'operations_performed': [
            'Missing value removal',
            'Duplicate removal', 
            'Short query filtering (<10 chars)',
            'Edge case handling',
            'Whitespace standardization'
        ],
        'records_removed': original_count - len(df_cleaned),
        'retention_rate': len(df_cleaned) / original_count,
        'cleaning_log': cleaning_log
    },
    'feature_engineering': {
        'total_features_created': len([col for col in df_cleaned.columns if col not in ['query', 'label']]),
        'feature_categories': {
            'query_structure': [
                'query_length', 'word_count', 'avg_word_length'
            ],
            'character_analysis': [
                'special_char_count', 'special_char_ratio', 'numeric_char_count', 
                'numeric_char_ratio', 'uppercase_count', 'lowercase_count', 
                'uppercase_ratio', 'alphabetic_ratio'
            ],
            'quote_manipulation': [
                'single_quote_count', 'double_quote_count', 'total_quote_count', 'quote_ratio'
            ],
            'comment_injection': [
                'has_sql_comment', 'has_hash_comment', 'has_multiline_comment_start',
                'has_multiline_comment_end', 'total_comment_indicators'
            ],
            'union_injection': [
                'has_union', 'has_union_select', 'has_union_all'
            ],
            'sql_keywords': [f'has_{kw.lower()}' for kw in sql_keywords.keys()] + [
                'sql_keyword_count', 'sql_keyword_density'
            ],
            'boolean_injection': [
                'has_or_condition', 'has_and_condition', 'has_equals_pattern', 'has_tautology'
            ],
            'advanced_patterns': [
                'has_semicolon', 'semicolon_count', 'has_drop_table', 
                'has_information_schema', 'has_system_functions',
                'parentheses_count', 'parentheses_ratio'
            ],
            'outlier_indicators': outlier_indicator_features
        },
        'eda_recommendations_implemented': [
            'EDA Recommendation #1: Query length features',
            'EDA Recommendation #3: SQL keyword density features', 
            'EDA Recommendation #4: Special character features',
            'EDA Recommendation #7: Short query filtering',
            'EDA Recommendation #8: Comment injection features',
            'EDA Recommendation #9: UNION-based injection features',
            'EDA Finding #4: Quote manipulation pattern features'
        ]
    },
    'outlier_analysis': {
        'approach': 'Security-focused (retention)',
        'outliers_removed': 0,
        'outliers_flagged': len(outlier_indicator_features),
        'rationale': 'Outliers retained as they often represent sophisticated attacks',
        'outlier_summary': outlier_summary
    },
    'final_dataset': {
        'total_records': len(df_cleaned),
        'total_columns': len(df_cleaned.columns),
        'total_features': len([col for col in df_cleaned.columns if col not in ['query', 'label']]),
        'memory_usage_mb': df_cleaned.memory_usage(deep=True).sum() / 1024**2,
        'class_distribution': dict(df_cleaned['label'].value_counts().sort_index()) if 'label' in df_cleaned.columns else {},
        'class_balance_ratio': balance_ratio_final
    },
    'validation_results': validation_results,
    'next_steps': {
        'ready_for_cnn_preprocessing': True,
        'recommended_next_task': 'Task 5: CNN-Specific Data Preprocessing',
        'notes': 'Dataset is cleaned and feature-engineered for advanced ML modeling'
    }
}

# Save the cleaned dataset
print(" Saving cleaned dataset...")
cleaned_dataset_path = os.path.join(data_processed_path, 'cleaned_sql_injection_dataset.csv')
df_cleaned.to_csv(cleaned_dataset_path, index=False)
print(f"    Dataset saved: {cleaned_dataset_path}")
print(f"    File size: {os.path.getsize(cleaned_dataset_path) / 1024**2:.1f} MB")

# Save preprocessing metadata
print("\n Saving metadata...")
metadata_path = os.path.join(data_processed_path, 'task4_preprocessing_metadata.json')
with open(metadata_path, 'w') as f:
    json.dump(preprocessing_metadata, f, indent=4, default=str)
print(f"    Metadata saved: {metadata_path}")

# Save feature documentation
feature_documentation = {
    'feature_names': {
        'all_features': [col for col in df_cleaned.columns if col not in ['query', 'label']],
        'numerical_features': list(df_cleaned.select_dtypes(include=[np.number]).columns),
        'categorical_features': [col for col in df_cleaned.columns if col.startswith('has_')],
        'outlier_indicators': [col for col in df_cleaned.columns if col.startswith('is_')]
    },
    'feature_descriptions': {
        'query_length': 'Number of characters in the SQL query',
        'word_count': 'Number of words in the SQL query', 
        'special_char_ratio': 'Ratio of special characters to total characters',
        'quote_ratio': 'Ratio of quote characters to total characters',
        'sql_keyword_density': 'Ratio of SQL keywords to total words',
        'total_comment_indicators': 'Total count of comment pattern indicators'
    },
    'usage_notes': {
        'for_cnn': 'All features ready for CNN input after normalization',
        'for_traditional_ml': 'Features can be used directly with traditional ML algorithms',
        'security_focus': 'Features designed specifically for SQL injection detection'
    }
}

feature_doc_path = os.path.join(data_processed_path, 'feature_documentation.json')
with open(feature_doc_path, 'w') as f:
    json.dump(feature_documentation, f, indent=4)
print(f"    Feature documentation saved: {feature_doc_path}")

# Generate executive summary report
executive_summary = f"""
DATASET PREPROCESSING & CLEANING
EXECUTIVE SUMMARY REPORT
{'='*60}
TRANSFORMATION OVERVIEW
{'='*60}

Original Dataset:      {original_count:,} records
Final Dataset:         {len(df_cleaned):,} records  
Data Retention:        {len(df_cleaned)/original_count*100:.1f}%
Features Created:      {len([col for col in df_cleaned.columns if col not in ['query', 'label']])}

{'='*60}
KEY ACHIEVEMENTS
{'='*60}

=> DATA QUALITY ENHANCEMENT
   • Removed missing values and duplicates
   • Filtered short queries (<10 chars) per EDA recommendation
   • Standardized text formatting and whitespace
   • Achieved {validation_results['overall_quality']} quality rating

=> COMPREHENSIVE FEATURE ENGINEERING  
   • Query structure analysis (3 features)
   • Character pattern analysis (8 features)
   • SQL injection pattern detection (20+ features)
   • Security-focused outlier indicators (8 features)
   • Total: {len([col for col in df_cleaned.columns if col not in ['query', 'label']])} engineered features

=> EDA RECOMMENDATIONS IMPLEMENTED
   • All 7 major EDA recommendations successfully applied
   • Quote manipulation patterns (67,683 cases) - addressed
   • Comment injection detection - implemented
   • UNION-based attack patterns - captured
   • SQL keyword density analysis - completed

=> SECURITY-FOCUSED APPROACH
   • Preserved attack outliers for model learning
   • Created domain-specific injection pattern features
   • Maintained class balance throughout processing
   • Built features specifically for cybersecurity context

{'='*60}
FINAL QUALITY METRICS
{'='*60}

Class Balance:         {balance_ratio_final:.2f}:1 (Excellent)
Missing Values:        {validation_results['missing_values']} 
Duplicate Records:     {validation_results['duplicates']}
Feature Quality:       {len([col for col in df_cleaned.columns if col not in ['query', 'label']])} features engineered
Memory Efficiency:     {df_cleaned.memory_usage(deep=True).sum() / 1024**2:.1f} MB"""

{'='*60}


# Save executive summary
summary_path = os.path.join(reports_path, 'task4_executive_summary.txt')
with open(summary_path, 'w') as f:
    f.write(executive_summary)
print(f"    Executive summary saved: {summary_path}")

# Display completion status
print("\n DATA SAVING COMPLETED SUCCESSFULLY!")
print(f"=" * 60)
print(f" Dataset Quality: {validation_results['overall_quality']}")
print(f" Records Processed: {len(df_cleaned):,}")
print(f"Features Engineered: {len([col for col in df_cleaned.columns if col not in ['query', 'label']])}")
print(f" Files Generated: 3")
print(f" Data Retention: {len(df_cleaned)/original_count*100:.1f}%")

print(f"\n OUTPUT STRUCTURE:")
print(f"    data/processed/")
print(f"   ├──  cleaned_sql_injection_dataset.csv")
print(f"   ├──  task4_preprocessing_metadata.json") 
print(f"   └──  feature_documentation.json")
print(f"    reports/")
print(f"   └──  task4_executive_summary.txt")



 STEP 6: SAVING CLEANED DATASET & METADATA:
 Missing variables: ['df_cleaned', 'original_count', 'data_processed_path', 'reports_path']
Please run the variable definition cell first!
 Saving cleaned dataset...
    Dataset saved: c:\Users\nisha\OneDrive\Desktop\Major-Project\Malicious-Query-detection-and-prevention\data\processed\cleaned_sql_injection_dataset.csv
    File size: 78.2 MB

 Saving metadata...
    Metadata saved: c:\Users\nisha\OneDrive\Desktop\Major-Project\Malicious-Query-detection-and-prevention\data\processed\task4_preprocessing_metadata.json
    Feature documentation saved: c:\Users\nisha\OneDrive\Desktop\Major-Project\Malicious-Query-detection-and-prevention\data\processed\feature_documentation.json
    Executive summary saved: c:\Users\nisha\OneDrive\Desktop\Major-Project\Malicious-Query-detection-and-prevention\reports\task4_executive_summary.txt

 DATA SAVING COMPLETED SUCCESSFULLY!
 Dataset Quality: EXCELLENT 
 Records Processed: 132,397
Features Engineered: 54
 